<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=8069f71e9b4592a0aa2e7c4ece3c0e636e7b3e82f1fb42a30da504e7a5a8d261
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2


    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-16 13:21:09
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.41 C
-------------------
Today PnL: -54.15 K (-0.38%)
Current PnL: -26.51 L (-17.13%)
CY Booked + Current PnL: -12.22 L (-7.9%)
-------------------
Total profit:  1.41 L
Total loss:  -27.93 L
-------------------
Total Booked + Current PnL: 14.76 L (11.54%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.17%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.15 L (64.86%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.28%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-1.15,-18.24,22.42,0.09,18474.0,-18378.0,82399.0,91.25,34.0,M-SC,2.33,253.0,-0.99,0.59,8.25,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,0.13,8.79,10.02,19.69,20902.0,16863.0,208605.0,-1.77,58.0,X-MC,2.53,78.0,0.81,1.49,16.19,XY25,NTT,AC
39,INDIGOPNTS,1408.00,-1.88,-13.34,15.41,0.02,23312.0,-23279.0,151280.0,130.47,49.0,M-SC,14.79,234.0,-1.00,1.08,31.59,OX40N,NTT,PAINTS
50,MASFIN,398.61,-1.99,-2.02,24.57,22.05,23587.0,-1980.0,96000.0,-15.54,54.0,H-SC,6.89,164.0,-0.08,0.69,39.54,XR,ATH,FINANCE
43,ITC,452.00,-0.04,-1.44,12.40,10.78,29371.0,-3452.0,236866.0,-41.15,46.0,X-LC,1.95,5.0,-0.12,1.69,3.72,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,CERA,9475.00,3.00,-30.24,77.60,23.89,95219.0,-53198.0,122705.0,-32.70,39.0,H-SC,10.58,157.0,-0.56,0.88,5.97,OX40N,NTT,CERAMICS
6,ASIANTILES,137.00,2.33,-16.59,117.81,81.67,92184.0,-15562.0,78248.0,7033.33,46.0,L-SC,12.38,271.0,-0.17,0.56,51.53,XR,NTT,CERAMICS
25,FINCABLES,1641.55,2.24,-1.81,111.74,107.91,161992.0,-2672.0,144972.0,-20.15,58.0,M-SC,12.54,216.0,-0.02,1.04,8.09,OX40N,ATH,CABLES
44,JCHAC,2282.00,1.45,-34.00,51.50,-0.01,43442.0,-43452.0,84353.0,17160.87,38.0,M-SC,7.33,233.0,-1.00,0.60,7.96,OX40N,NTT,AC
23,DMART,5391.45,1.25,-6.89,39.20,29.60,91100.0,-17202.0,232398.0,-20.63,39.0,X-LC,5.68,19.0,-0.19,1.66,13.79,X40N,ATH,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RAJOOENG,143.10,-2.86,-35.72,117.18,39.61,77210.0,-36610.0,65890.0,-56.61,34.0,H-SC,28.73,135.0,-0.47,0.47,3.81,AR,ATH,MISC
63,SFL,1287.00,-2.25,-44.26,127.18,26.63,185882.0,-116061.0,146157.0,10.73,25.0,M-SC,11.68,239.0,-0.62,1.04,0.00,XY24,NTT,MISC
32,HCLTECH,1922.01,-2.11,6.98,16.59,24.73,42937.0,16890.0,258814.0,11.91,55.0,X-LC,6.83,7.0,0.39,1.85,23.28,X40,ATH,IT
21,DEN,75.00,-2.06,-40.65,139.39,42.07,69873.0,-34336.0,50128.0,78.76,47.0,M-SC,7.80,235.0,-0.49,0.36,6.46,AR,NTT,ENTERTAINMENT
50,MASFIN,398.61,-1.99,-2.02,24.57,22.05,23587.0,-1980.0,96000.0,-15.54,54.0,H-SC,6.89,164.0,-0.08,0.69,39.54,XR,ATH,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,2.24,-1.81,111.74,107.91,161992.0,-2672.0,144972.0,-20.15,58.0,M-SC,12.54,216.0,-0.02,1.04,8.09,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.88,-13.34,15.41,0.02,23312.0,-23279.0,151280.0,130.47,49.0,M-SC,14.79,234.0,-1.00,1.08,31.59,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-1.15,-18.24,22.42,0.09,18474.0,-18378.0,82399.0,91.25,34.0,M-SC,2.33,253.0,-0.99,0.59,8.25,OX40N,NTT,DURABLES
66,SIS,528.00,0.14,-24.12,59.81,21.26,50391.0,-26780.0,84252.0,1987.56,50.0,H-SC,5.12,166.0,-0.53,0.60,13.83,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.34,-24.71,50.71,13.47,102962.0,-66627.0,203040.0,-69.09,36.0,H-SC,5.19,158.0,-0.65,1.45,3.24,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.63,6.68,61.48,72.27,111137.0,11322.0,180770.0,-8.35,65.0,M-LC,2.46,99.0,0.1,1.29,13.27,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.63,6.68,61.48,72.27,111137.0,11322.0,180770.0,-8.35,65.0,M-LC,2.46,99.0,0.10,1.29,13.27,XR,NTT,IT
25,FINCABLES,1641.55,2.24,-1.81,111.74,107.91,161992.0,-2672.0,144972.0,-20.15,58.0,M-SC,12.54,216.0,-0.02,1.04,8.09,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.59,-4.16,115.68,106.72,136746.0,-5125.0,118211.0,-52.54,34.0,H-SC,3.00,139.0,-0.04,0.85,18.25,AR,ATH,MISC
1,ABB,7934.00,-0.39,-3.41,50.71,45.57,128142.0,-8923.0,252696.0,-37.97,58.0,H-MC,4.55,120.0,-0.07,1.81,7.52,AR,NTT,ELECTRICAL
50,MASFIN,398.61,-1.99,-2.02,24.57,22.05,23587.0,-1980.0,96000.0,-15.54,54.0,H-SC,6.89,164.0,-0.08,0.69,39.54,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.35,-18.24,90.15,55.47,179325.0,-44368.0,198918.0,-27.60,25.0,X-MC,12.93,64.0,-0.25,1.42,1.92,X40N,ATH,FINANCE
53,PAGEIND,51605.07,-0.75,-9.48,41.52,28.11,75701.0,-19085.0,182325.0,-34.38,28.0,X-MC,12.93,55.0,-0.25,1.30,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-0.62,-29.29,73.91,22.98,118060.0,-66163.0,159735.0,-26.74,29.0,X-LC,6.81,3.0,-0.56,1.14,0.47,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-1.18,-38.85,121.10,35.20,95286.0,-49986.0,78684.0,3.10,31.0,X-SC,16.13,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
8,AWL,485.00,-0.95,-22.78,98.77,53.50,230154.0,-68731.0,233020.0,-62.61,31.0,X-MC,2.79,58.0,-0.30,1.67,3.50,XY24,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.53,-1.57,26.46,24.48,65593.0,-3960.0,247896.0,-81.32,43.0,X-SC,0.31,86.0,-0.06,1.77,21.20,X40N,NTT,MISC
43,ITC,452.0,-0.04,-1.44,12.40,10.78,29371.0,-3452.0,236866.0,-41.15,46.0,X-LC,1.95,5.0,-0.12,1.69,3.72,X40,NTT,FMCG
33,HINDUNILVR,2922.0,-0.32,-11.06,27.81,13.67,63579.0,-28443.0,228620.0,-17.78,32.0,X-LC,2.14,14.0,-0.45,1.63,7.93,XY25,NTT,FMCG
36,ICICIPRULI,790.0,-0.46,6.82,22.39,30.73,47116.0,13428.0,210433.0,-18.17,62.0,X-MC,2.30,75.0,0.28,1.50,20.45,X40,ATH,INSURANCE
78,UNITDSPR,1644.0,0.43,6.41,13.49,20.77,33612.0,15015.0,249159.0,-2.78,57.0,X-MC,2.45,62.0,0.45,1.78,13.77,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.43,-25.61,120.68,64.15,213604.0,-60951.0,177000.0,-56.31,34.0,X-SC,2.60,82.0,-0.29,1.27,0.00,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-1.18,-38.85,121.10,35.20,95286.0,-49986.0,78684.0,3.10,31.0,X-SC,16.13,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
53,PAGEIND,51605.07,-0.75,-9.48,41.52,28.11,75701.0,-19085.0,182325.0,-34.38,28.0,X-MC,12.93,55.0,-0.25,1.30,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-0.62,-29.29,73.91,22.98,118060.0,-66163.0,159735.0,-26.74,29.0,X-LC,6.81,3.0,-0.56,1.14,0.47,XY24,NTT,AUTO
59,RELAXO,1176.00,-0.42,-47.87,192.14,52.31,145410.0,-69481.0,75679.0,-44.54,41.0,X-SC,6.93,91.0,-0.48,0.54,1.30,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.38,-29.26,101.92,42.84,46869.0,-19020.0,45986.0,-53.72,44.0,X-SC,35.52,83.0,-0.41,0.33,4.04,XY24,NTT,MISC
59,RELAXO,1176.00,-0.42,-47.87,192.14,52.31,145410.0,-69481.0,75679.0,-44.54,41.0,X-SC,6.93,91.0,-0.48,0.54,1.30,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.18,-38.85,121.10,35.20,95286.0,-49986.0,78684.0,3.10,31.0,X-SC,16.13,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.58,-4.69,30.18,24.08,36517.0,-5951.0,120999.0,-12.39,32.0,X-SC,5.12,89.0,-0.16,0.87,14.01,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.03,-16.36,71.36,43.32,97207.0,-26652.0,136220.0,-28.18,38.0,X-SC,4.74,90.0,-0.27,0.97,4.59,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-0.91,-11.99,37.15,20.71,123666.0,-45354.0,332883.0,-24.27,58.0,X-LC,4.46,1.0,-0.37,2.38,11.23,X40,ATH,IT
41,INFY,1972.00,-0.70,8.61,23.60,34.24,80203.0,26954.0,339842.0,-15.29,60.0,X-LC,4.02,2.0,0.34,2.43,17.61,X40,NTT,IT
75,TMPV,600.00,-0.62,-29.29,73.91,22.98,118060.0,-66163.0,159735.0,-26.74,29.0,X-LC,6.81,3.0,-0.56,1.14,0.47,XY24,NTT,AUTO
81,VBL,671.64,0.50,-3.04,39.92,35.67,122251.0,-9602.0,306240.0,-14.52,58.0,X-LC,4.12,4.0,-0.08,2.19,10.39,X40N,ATH,FMCG
43,ITC,452.00,-0.04,-1.44,12.40,10.78,29371.0,-3452.0,236866.0,-41.15,46.0,X-LC,1.95,5.0,-0.12,1.69,3.72,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-1.05,-37.81,113.70,32.89,54589.0,-29196.0,48011.0,-693.00,52.0,L-MC,5.95,259.0,-0.53,0.34,32.22,XR,NTT,BANKS
6,ASIANTILES,137.00,2.33,-16.59,117.81,81.67,92184.0,-15562.0,78248.0,7033.33,46.0,L-SC,12.38,271.0,-0.17,0.56,51.53,XR,NTT,CERAMICS
50,MASFIN,398.61,-1.99,-2.02,24.57,22.05,23587.0,-1980.0,96000.0,-15.54,54.0,H-SC,6.89,164.0,-0.08,0.69,39.54,XR,ATH,FINANCE
13,BSOFT,831.70,-1.72,-22.54,94.66,50.79,103131.0,-31704.0,108949.0,-0.66,63.0,H-SC,6.33,151.0,-0.31,0.78,26.85,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.88,-13.34,15.41,0.02,23312.0,-23279.0,151280.0,130.47,49.0,M-SC,14.79,234.0,-1.00,1.08,31.59,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.7,-1.72,-22.54,94.66,50.79,103131.0,-31704.0,108949.0,-0.66,63.0,H-SC,6.33,151.0,-0.31,0.78,26.85,XR,ATH,IT
84,WIPRO,420.0,-0.63,6.68,61.48,72.27,111137.0,11322.0,180770.0,-8.35,65.0,M-LC,2.46,99.0,0.10,1.29,13.27,XR,NTT,IT
41,INFY,1972.0,-0.70,8.61,23.60,34.24,80203.0,26954.0,339842.0,-15.29,60.0,X-LC,4.02,2.0,0.34,2.43,17.61,X40,NTT,IT
1,ABB,7934.0,-0.39,-3.41,50.71,45.57,128142.0,-8923.0,252696.0,-37.97,58.0,H-MC,4.55,120.0,-0.07,1.81,7.52,AR,NTT,ELECTRICAL
27,GLAXO,3466.2,0.54,4.05,33.73,39.15,69940.0,8068.0,207352.0,-16.38,54.0,X-MC,10.69,60.0,0.12,1.48,34.22,X40,ATH,PHARMA


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.26
1,25,44.86
2,50,76.64


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.97
MC    30.98
LC    24.05
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.73
X40      22.95
X40N     13.07
XR        9.76
XY25      9.08
AR        9.07
OX40N     7.58
SR        0.99
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.50
H-SC    23.81
X-LC    20.62
M-SC    11.69
X-SC     8.04
H-MC     4.82
L-SC     1.43
M-MC     1.32
M-LC     1.29
H-LC     1.13
L-LC     1.01
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.41,-7.10,41.54
IT,12.99,-15.69,73.85
FINANCE,9.49,-20.17,72.44
MISC,7.24,-28.20,80.64
ELECTRICAL,6.04,-11.24,51.99
PAINTS,5.84,-12.95,29.72
INSURANCE,4.77,-1.51,36.25
PHARMA,4.12,-1.80,33.84
AUTO,2.81,-33.53,79.83


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3186088.0,21
AR,1305537.0,10
XR,1296769.0,13
X40,1009783.0,14
X40N,855241.0,9
OX40N,731731.0,10
XY25,371125.0,6
SR,281739.0,2
MH,76915.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3651522.0,25
M-SC,1424375.0,15
X-MC,1398040.0,16
X-LC,840346.0,11
X-SC,794295.0,8
H-MC,404556.0,3
L-SC,261208.0,3
M-LC,111137.0,1
H-LC,76062.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1275827.0      6
           AR         894107.0      5
M-SC       XY24       819628.0      6
H-SC       XR         781351.0      7
X-MC       X40        485359.0      7
           XY24       401714.0      3
H-SC       OX40N      341583.0      4
X-LC       X40        335329.0      5
X-MC       X40N       333680.0      4
X-SC       X40N       308210.0      3
M-SC       OX40N      304930.0      5
X-SC       XY24       296990.0      3
H-SC       SR         281739.0      2
X-LC       X40N       213351.0      2
H-MC       AR         209253.0      2
X-LC       XY24       196626.0      2
H-MC       XY24       195303.0      1
X-SC       X40        189095.0      2
X-MC       XY25       177287.0      2
L-SC       XR         175990.0      2
M-SC       XR         173702.0      2
           AR         126115.0      2
M-LC       XR         111137.0      1
X-LC       XY25        95040.0      2
L-SC       OX40N       85218.0      1
H-SC       MH          76915.0      1
H-LC       AR          76062.0      1
M-MC       XY25        60615.0      1
L-MC       XR          54589.0      1
L-LC       XY25        38183.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
